---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **JsonOutputParser**

* 사용자가 원하는 `JSON 스키마`를 지정할 수 있게 해주며, 그 스키마에 맞게 `LLM`에서 데이터를 조회하여 결과를 도출해줌

* `LLM`이 데이터를 정확하고 효율적으로 처리하여 원하는 형태의 `JSON`을 생성하기 위해서는, 모델의 용량이 충분해야 한다는 점을 참고

In [1]:
# 1_새 프롬프트 생성하기
from langsmith import Client
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langsmith import Client

import os
import json


# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langsmith import traceable

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

* 원하는 **`출력 구조 정의`**

In [4]:
# 원하는 데이터 구조 정의

class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

* `JsonOutputParser` → 파서 설정 → 프롬프트 템플릿 지시사항 주입

In [ ]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | gemini_lc | parser                                                     # 체인 구성

chain.invoke({"question": question})                                                    # 체인을 호출하여 쿼리 실행


<small>

* 셀 출력 (2.1s)

    ```json
    {'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 해수면 상승, 극심한 기상 이변, 생태계 파괴 등 심각한 결과를 초래합니다. 이는 인간 활동으로 인한 온실가스 배출 증가가 주된 원인입니다.',
    'hashtags': '#지구온난화 #기후변화 #환경문제 #온실가스 #지속가능성'}
    ```

### **`Pydantic`** 없이 사용하기

* `Pydantic` 없이도 사용 가능

* `JSON` 반환 요청 → 스키아가 어떻게 되어야 하는지에 대한 구체적인 정보는 제공하지 않음

In [ ]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | gemini_lc | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)

<small>

* 셀 출력 (1.0s)

    ```json
    {'description': '지구 온난화는 인간 활동으로 인해 대기 중 온실가스 농도가 증가하면서 지구의 평균 기온이 상승하는 현상을 말합니다. 이는 해수면 상승, 극한 기상 현상 증가, 생태계 변화 등 다양한 환경 문제를 야기합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지속가능성']}
    ```